In [35]:
import os
import io
import csv
from read_data import convert_pdf_to_txt
from PIL import Image
import pytesseract
from wand.image import Image as wi
import hashlib

In [36]:
pdfs = os.listdir('./pdfs')

In [34]:
start_with_header = 1

with open('out_louisiana.csv', 'w', newline='') as fcsv:
    for pdf in pdfs:
        with open('./pdfs/' + pdf, 'rb') as f:
            # text from pdf
            bfr = io.BufferedReader(f)
            pdf_text = convert_pdf_to_txt(bfr) 

            # recognized text (OCR) from pdf
            if len(pdf_text.strip()) <= 50:
                with wi(filename=pdf, resolution=200) as pdf_file:
                    pdfImage = pdf_file.convert('jpeg')
                    imageBlobs = []
                    for img in pdfImage.sequence:
                        with wi(image = img) as imgPage:
                            imageBlobs.append(imgPage.make_blob('jpeg'))

                recognized_text = []

                for imgBlob in imageBlobs:
                    im = Image.open(io.BytesIO(imgBlob))
                    text = pytesseract.image_to_string(im, lang = 'eng')
                    recognized_text.append(text)

                recognized_text = '\n\n\n'.join(recognized_text)

            pdf_text = pdf_text if len(pdf_text.strip()) > 50 else recognized_text
            
            writer = csv.writer(fcsv)
            
            if start_with_header:
                writer.writerow(['filename', 'text'])
                
            start_with_header = 0
            
            writer.writerow([pdf, pdf_text])